# Libraries

In [19]:
import datacube
acube = datacube.Datacube(app='boku', env='acube')
from acube_tools import query_data
from acube_tools import image_selection_byCloud

/app/opt/venv/lib/python3.8/site-packages/datacube/drivers/postgres/_connections.py:81: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  EngineUrl(


In [20]:
import matplotlib.pyplot as plt
import math
import datetime
from datetime import timedelta, date
from datacube.utils.cog import write_cog
import gdal
from pylab import *
import numpy as np
import os
import matplotlib.colors as clr
import itertools
import csv
from osgeo import ogr
import joblib

In [21]:
from acube_tools import get_long_lat
from acube_tools import VegetationIndex
from acube_tools import maskGeneration
from acube_tools import loadGeoTiff
from acube_tools import ind_VfoldCross
from acube_tools import RandomForestClassification
#from acube_tools import plot_confusion_matrix
from acube_tools import rf_save_results
from acube_tools import diff_emma
from acube_tools import writeout
from acube_tools import scale
#from Tools import write_tiff
#from Tools import RGBplot

In [22]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier

# User input data

- más adelante (en el acube hacer un identifier con el area y las fechas de las que sale el RF). Ejemplo:
- identifier = f'ID_dates_{start_date}-{end_date}_area_{latitude}-{longitude}'
- Acube returns B02, B03, B04, B05, B06, B07, B08, B8A, B11 and B12 for Sentinel 2
- num_images = 2 ##no lo uso pero lo puedo anyadir
- Aditional info
    - Flood event: Carinthia: 29-10-2018
    - Branau:    01-06-2016
    - Strobler:  27-07-2019

In [4]:
fdate = date(2019, 7, 27) # inserted by user
area = 'Carinthia'
subarea = 'Gail'
path_out = '/home/boku/jupyter-datacube-docker-singularity/results/'
identifier = f'ejemplo' 
cpercen = 20

In [28]:
#WARNING!! NO PUEDO DEFINIR EL AREA A PARTIR DE LA MASCARA
mask_tif = f'/home/boku/jupyter-datacube-docker-singularity/masks/temp/mask_{area}.tif' 
longitude, latitude = get_long_lat(mask_tif)
print('longitude', longitude)
print('latitude', latitude)

longitude (13.188319446101904, 13.270605126288197)
latitude (45.79732853116177, 45.96216938579245)


# Code

In [6]:
# Hardcoded variables
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8',' B8A', 'B11', 'B12']
features = ['NDVI', 'NDWI', 'NDMI', 'TCARI', 'SAVI', 'KNDVI', 'MSAVI2']
target_names=['no_flood','flood']
path_gpkg = '/home/boku/bmlrt/'
path_shapefiles = '/home/boku/jupyter-datacube-docker-singularity/shapefiles/'

In [29]:
day_b

'2019-06-05'

In [7]:
# Data after and before event:
day_b, s2_b, cloud_b, s2_geo_b = image_selection_byCloud (fdate, latitude, longitude, cpercen, 'before')
day_a, s2_a, cloud_a, s2_geo_a = image_selection_byCloud (fdate, latitude, longitude, cpercen, 'after')
print('Shape s2 array before event:', s2_b.shape)
print('Shape s2 array after event:', s2_a.shape)

----------------------------------
New Date: 2019-07-27
day 2019-07-27 is in acube_dates
Calculating cloud coverage...
No cloud coverage
sentinel 2 band-arrays full of zeros!
----------------------------------
New Date: 2019-07-26
day 2019-07-26 is not in acube_dates
----------------------------------
New Date: 2019-07-25
day 2019-07-25 is in acube_dates
Calculating cloud coverage...
Cloud coverage (%): 93.42616786693009
Cloud coverage more than 20%
----------------------------------
New Date: 2019-07-24
day 2019-07-24 is not in acube_dates
----------------------------------
New Date: 2019-07-23
day 2019-07-23 is in acube_dates
Calculating cloud coverage...
Cloud coverage (%): 93.44034797017399
Cloud coverage more than 20%
----------------------------------
New Date: 2019-07-22
day 2019-07-22 is not in acube_dates
----------------------------------
New Date: 2019-07-21
day 2019-07-21 is not in acube_dates
----------------------------------
New Date: 2019-07-20
day 2019-07-20 is in acub

End
Shape s2 array before event: (10, 1846, 680)
Shape s2 array after event: (10, 1846, 680)


In [8]:
s2_b.shape

(10, 1846, 680)

In [9]:
mask_tif = f'/home/boku/jupyter-datacube-docker-singularity/masks/mask_{area}_{subarea}.tif' #WARNING!! NO PUEDO DEFINIR EL AREA A PARTIR DE LA MASCARA

In [10]:
# Mask: flood - no flood -- WARNING!!! I CAN NOT USE EMMA FUNCTION
if os.path.isfile(mask_tif):
    print('the mask is generated')
else:
    maskGeneration(s2_b, s2_geo_b, area, subarea, path_gpkg, path_shapefiles, mask_tif)

the mask is generated


In [11]:
clas = loadGeoTiff(mask_tif)
Y_total = np.reshape(clas, (clas.shape[0]*clas.shape[1], 1)).ravel()

In [12]:
clas=np.transpose(clas)
clas.shape

(1846, 680)

In [13]:
# Add vegetation indices
vi_b = VegetationIndex(s2_b, bands, features)
vi_a = VegetationIndex(s2_a, bands, features)

In [14]:
# Get difference
data = vi_a - vi_b
print('Data shape:', data.shape)

Data shape: (1255280, 17)


In [15]:
# Clean all data (Nan): spectral, indices and mask
X_clean = data[~np.isnan(data).any(axis=1)]
Y_clean = Y_total[~np.isnan(data).any(axis=1)]

In [16]:
# Select training data:
list_of_train = ind_VfoldCross(Y_clean, 1500) # WARNING!!! THINK HOW TO PRODUCE THIS NUMBER

X_train = X_clean[list_of_train, :]
Y_train = Y_clean[list_of_train]

In [17]:
# Optimize and train the classifier:
Nt = [int(i) for i in np.round(np.linspace(50, 500, num=30))]
mtr = [int(i) for i in np.linspace(1, X_train.shape[1]-1, num=6)]

[cl_Final, param] = RandomForestClassification(X_train, Y_train,mtr, -1, 5, Nt)

Cross validation...
CV done!
Training!
Predicting!


In [23]:
# Select test data:
list_of_test = diff_emma(range(Y_clean.shape[0]), list_of_train)

X_test = X_clean[list_of_test, :]
Y_test = Y_clean[list_of_test]

In [24]:
# Predition in test samples:
Ypred = cl_Final.predict(X_test)

In [27]:
# Statistics:

OA = accuracy_score(Y_test, Ypred)*100
print('OA:', OA)

kappa = cohen_kappa_score(Y_test, Ypred)
print('kappa:', kappa)

CM = confusion_matrix(Y_test, Ypred)
print('CM:', CM)

OA1c = CM[1, 1] / (CM[1, 0] + CM[1, 1])
print('OA1c*100:', OA1c*100)
PEN1c = exp(-(CM[0, 1] / (-(CM[1, 0] + CM[1, 1]) / log(1 / 2))))
print('PEN1c:', PEN1c)
# PEN1c = exp(-(CM[0, 1] / (-(sum(CM)) / log(1 / 2))))
SR1c = OA1c - (1 - PEN1c)

print('SR1c*100:', SR1c*100)

OA: 100.0


/app/opt/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:638: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


kappa: nan
CM: [[1253780]]


IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
# Save results RF --WARNING!! ADD SAVE RF CLASSIFIER
rf_save_results(CM, OA1c, kappa, PEN1c, SR1c, target_names, path_out, identifier)

In [ ]:
# Prediction map:
base_array[np.isnan(base_array)] = 999999999
Ypred = cl_Final.predict(base_array)
ProbPred = cl_Final.predict_proba(base_array)
Ypred[(base_array == 999999999).any(axis=1)] = 0
Y_clas = np.reshape(Ypred, (y1.shape[1], y1.shape[2]))

Prob_clas = np.reshape(ProbPred, (y1.shape[1], y1.shape[2], 2))

In [ ]:
# Scale the flood prob between 0 and 1:
Prob_flood_ns = Prob_clas[np.where(Y_clas == 1)[0], np.where(Y_clas == 1)[1], 1]
Prob_flood_ns = np.reshape(Prob_flood_ns, (Prob_flood_ns.shape[0], 1))
Prob_flood = scale(Prob_flood_ns)
Prob_fl = np.zeros((y1.shape[1], y1.shape[2]))

Prob_fl[np.where(Y_clas == 1)[0], np.where(Y_clas == 1)[1]] = np.reshape(Prob_flood[0], (Prob_flood[0].shape[0]))

In [ ]:
# Plotting classification and probablity results:
rcParams.update({'figure.autolayout': True})
colors1 = ['#000080', '#FFFFFF', '#FFFF00']
colors1 = [colors1[int(i-1)] for i in np.unique(Y_clas[~np.isnan(Y_clas)])]

cmap = clr.ListedColormap(colors1)

cmap = cm.get_cmap(cmap, len(np.unique(Y_clas[~np.isnan(Y_clas)])))

f = plt.figure()
plt.imshow(clas, cmap=cmap)
plt.axis('off')

f = plt.figure()
plt.imshow(Y_clas, cmap=cmap)
plt.axis('off')

f = plt.figure()
plt.imshow(Prob_fl)
plt.axis('off')
plt.show()

In [ ]:
writeout(clas, path_out, identifier, s2_geo_b['lon'], s2_geo_b['lat'])

In [ ]:
'''# Save the classification map in tiff:
tifsrc = gdal.Open(root_image)

if 0 == 1:
    write_tiff(clas, tifsrc, path_out, name_out)

    name = 'prediction.tif'
    write_tiff(Y_clas, tifsrc, path_out, name_out)

    name = 'probability.tif'
    write_tiff(Prob_fl, tifsrc, path_out, name_out)'''